### A fuzzy-KNN Algorithm

One major issue classification algorithms face while extracting rules are giving equal importance to every sample in the dataset. The paper “A Fuzzy K-Nearest Neighbor Algorithm” by James M. Keller, Michael R. Gray, and James A. Givens, JR. introduces the theory of fuzzy sets and incorporates the fuzzy search techniques into the KNN algorithm. The fuzzy algorithm outperforms the generic KNN in terms of mistake rate and holds up well against more conventional, complex pattern recognition techniques. 
</br> </br>
The issue with the base KNN classifier is that each sample is given an equal weight. This creates issues in certain situations when sample sets overlap. Equal weight is given to typical vectors and the actual representatives of the cluster. Another issue is that when one of the vectors is classified, the membership strength cannot be determined. Hence the fuzzy set theory is introduced to combat all these issues

In [61]:
import operator
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

### Load the dataset

In [62]:
dataset = load_breast_cancer()
x = dataset.data
y = dataset.target

### Split the dataset into train and test

In [63]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, random_state=56, test_size=0.3)

### Fuzzy KNN

The fuzzy K-nearest neighbor algorithm assigns class membership to a sample vector rather than assigning the vector to a particular class. The advantage is that no arbitrary assignments are made by the algorithm. 

In [64]:
class FuzzyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, plot=False):
        self.k = k
        self.plot = plot
        
        
    def fit(self, X, y=None):
        self.X = X
        self.y = y
        
        self.xdim = len(self.X[0])
        self.n = len(y)
        #Initialize the classes which we have to predict
        classes = list(set(y))
        classes.sort()
        self.classes = classes
        #Create the dataframe 
        self.df = pd.DataFrame(self.X)
        self.df['y'] = self.y
        #Get the fuzzy memberships for all the data points
        self.memberships = self._compute_memberships()
        #Update the memberships for all the data points
        self.df['membership'] = self.memberships
        
        self.fitted_ = True
        return self
    
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('predict() called before fit()')
        y_pred = list()
        
        for x in X:
            #get the k nearest neighbours
            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            
            votes = dict()
            for c in self.classes:
                den = 0
                 # We iterate over all the neighbours and find its distance from current point
                for n in range(self.k):
                    dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                    den += 1 / (dist ** 2)
                #Once the distance is computed, sum of vote of neighbour determines the predicted class
                # each vote of a neighbour is calculated using the distance from point and the fuzzy membership of the predicted class 
                neighbors_votes = []
                for n in range(self.k):
                    dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                    num = (neighbors.iloc[n].membership[c]) / (dist ** 2)
                    
                    vote = num/den
                    neighbors_votes.append(vote)
                votes[c] = np.sum(neighbors_votes)
            #class with max votes is predicted   
            pred = max(votes.items(), key=operator.itemgetter(1))[0]
            y_pred.append((pred, votes))
            
        return y_pred

    def _find_k_nearest_neighbors(self, df, x):
         #find K nearest neighbours from point x (using euclidean distance)
        X = df.iloc[:,0:self.xdim].values
        df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]
        #Sort the points by distance for ease of computation later
        df.sort_values(by='distances', ascending=True, inplace=True)
        neighbors = df.iloc[0:self.k]
        
        return neighbors

                
    def _get_counts(self, neighbors):
        groups = neighbors.groupby('y')
        #Count of each class in neighbours as a dictionary which will be used for voting
        counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}
        return counts
        
        
    def _compute_memberships(self):
        memberships = []
        for i in range(self.n):
            x = self.X[i]
            y = self.y[i]
            #Get the k nearest neighbours and counts of each classes
            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            counts = self._get_counts(neighbors)

            #compute the membership of each class
            membership = dict()
            for c in self.classes:
                try:
                    #uci is the degree of membership of each vector in class 'classes'
                    uci = 0.49 * (counts[c] / self.k)
                    if c == y:
                        uci += 0.51
                    membership[c] = uci
                except:
                    #if no neighbours found
                    membership[c] = 0
                    
            memberships.append(membership)
        return memberships

### Comparing Fuzzy KNN with baseline KNN classifier

In [65]:
analysis = {
    'model' : list(),
    'Training time' : list(),
    'Prediction time' : list(),
    'Accuracy' : list()
}

In [66]:
def add_classifier(d):
    for i in analysis.keys():
        analysis[i].append(d[i])

In [67]:
baseline_knn = {x : None for x in analysis.keys()}
baseline_knn['model'] = 'Baseline KNN'
knn = KNeighborsClassifier(n_neighbors=5)
start_training_time = datetime.now()
knn.fit(x_train, y_train)
end_training_time = datetime.now()
baseline_knn['Training time'] = (end_training_time - start_training_time).total_seconds()
start_prediction_time = datetime.now()
y_pred = knn.predict(x_test)
end_prediction_time = datetime.now()
baseline_knn['Prediction time'] = (end_prediction_time - start_prediction_time).total_seconds()
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
baseline_knn['Accuracy'] = accuracy
add_classifier(baseline_knn)


Accuracy: 0.9122807017543859


In [68]:
fuzzyknn = FuzzyKNN(k = 5)
fuzzy_knn_classifier = {x : None for x in analysis.keys()}
fuzzy_knn_classifier['model'] = 'Fuzzy KNN'
start_training_time = datetime.now()
fuzzyknn.fit(x_train, y_train)
end_training_time = datetime.now()
fuzzy_knn_classifier['Training time'] = (end_training_time - start_training_time).total_seconds()

start_prediction_time = datetime.now()
y_pred = fuzzyknn.predict(x_test)
end_prediction_time = datetime.now()
fuzzy_knn_classifier['Prediction time'] = (end_prediction_time - start_prediction_time).total_seconds()

print(y_pred)

y_actual_pred = list()
for (pred_class, fuzzy_membership_votes) in y_pred : 
    y_actual_pred.append(pred_class)
score = accuracy_score(y_test, y_actual_pred)
print(score)
fuzzy_knn_classifier['Accuracy'] = score
add_classifier(fuzzy_knn_classifier)


[(0, {0: 1.0, 1: 0.0}), (1, {0: 0.0, 1: 0.9999999999999999}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.0, 1: 1.0}), (0, {0: 0.974758057090137, 1: 0.025241942909863094}), (1, {0: 0.0, 1: 0.9999999999999999}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.1629551356975058, 1: 0.837044864302494}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.0, 1: 0.9999999999999999}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.21337711409128932, 1: 0.7866228859087109}), (0, {0: 1.0, 1: 0.0}), (1, {0: 0.0744241872608169, 1: 0.9255758127391833}), (0, {0: 1.0, 1: 0.0}), (1, {0: 0.0, 1: 0.9999999999999999}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.0, 1: 1.0000000000000002}), (0, {0: 1.0, 1: 0.0}), (1, {0: 0.01242821188944077, 1: 0.987571788110559}), (1, {0: 0.0, 1: 1.0000000000000002}), (1, {0: 0.0, 1: 1.0}), (1, {0: 0.42702919551415996, 1: 0.5729708044858401}), (1, {0: 0.0, 1: 0.9999999999999999}), (0, {0: 1.0, 1: 0.0}), (1, {0: 0.28509034805246003, 1: 0.7149096519475399}), (0, {0: 0.9999999999999999, 1: 0.0}), (1, {0: 0.0, 1: 1.0000000000000002}), (1, {0:

The y_pred depicts the tuple - (predicted_class, fuzzy_membership_votes)
where predicted_class is the class predicted for the datapoint and fuzzy_membership_votes shows the votes given by each of the k nearest neighbour

In [69]:
analysis_df = pd.DataFrame(analysis)
analysis_df

,model,Training time,Prediction time,Accuracy
0,Baseline KNN,0.000000,0.016011,0.912281
1,Fuzzy KNN,3.937506,2.397052,0.923977
